Market Basket Analysis

In [9]:
!pip3 install efficient-apriori
!pip3 install pymongo

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from efficient_apriori import apriori
from IPython.display import display

In [11]:
jobs_df = pd.read_csv('jobs_processed.csv')

In [12]:
jobs_df.describe()

,Unnamed: 0
count,427.000000
mean,213.000000
std,123.408536
min,0.000000
25%,106.500000
50%,213.000000
75%,319.500000
max,426.000000


In [13]:
jobs_df.shape

(427, 6)

In [14]:
jobs_df.head()

,Unnamed: 0,company,title,stacks,salary,description
0,0,Accedo.tv,Software Engineer (Android),"['TDD', 'HTML', 'React Native', 'Android', 'Ko...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
1,1,Accedo.tv,Software Engineer (Android),"['TDD', 'HTML', 'React Native', 'Android', 'Ko...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
2,2,Accedo.tv,Software Engineer (Android),"['TDD', 'HTML', 'React Native', 'Android', 'Ko...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
3,3,GXS Bank,"Software Engineer, Mobile (IOS)","['TDD', 'CI', 'RxSwift', 'MVVM', 'iOS', 'Swift']","['5,500', '8,250']",About The Team\n\nWhen people talk about Digib...
4,4,TikTok,Android Software Engineer - TikTok Live,"['Java', 'Android', 'Kotlin']","['6,787', '13,620']","Why Join Us\nAt TikTok, our people are humble,..."


In [15]:
def str_to_list(string):
    return ast.literal_eval(string)

jobs_df['stacks'] = jobs_df['stacks'].apply(str_to_list)


In [16]:
jobs_df

,Unnamed: 0,company,title,stacks,salary,description
0,0,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
1,1,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
2,2,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
3,3,GXS Bank,"Software Engineer, Mobile (IOS)","[TDD, CI, RxSwift, MVVM, iOS, Swift]","['5,500', '8,250']",About The Team\n\nWhen people talk about Digib...
4,4,TikTok,Android Software Engineer - TikTok Live,"[Java, Android, Kotlin]","['6,787', '13,620']","Why Join Us\nAt TikTok, our people are humble,..."
...,...,...,...,...,...,...
422,422,Infogain,Software Engineer(2-3 years)-C,"[TDD, C++, C, Linux, Python]",0,"FW build and release, defects triaging, test s..."
423,423,UBS,Technical Engineer,"[GitLab, Oracle, play, UNIX, SQL]","['7,335', '12,765']",Your role\n\nDo you have the know-how to apply...
424,424,DXC Technology,Junior Java Developer,"[Oracle, Visual Studio, Eclipse, VB.Net, SQL, ...","['3,423', '4,886']",Applies specialized knowledge to conceptualize...
425,425,DXC Technology,Full stack/Node JS Developer,"[GitLab, Visual Studio, Git, Node.js, npm, Jav...","['3,786', '5,614']",Scope Of Work\n\nReport to the Authority’s Pr...


In [17]:
records = []

for index, row in jobs_df.iterrows():
    records.append([stack for stack in row['stacks']])

print(len(records))

print(records[0:10])

427
[['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'CI', 'RxSwift', 'MVVM', 'iOS', 'Swift'], ['Java', 'Android', 'Kotlin'], ['C++', 'Java', 'C', 'C#'], ['Graph', 'C++', 'Go', 'Linux', 'Python'], ['C++', 'Java', 'Android', 'C#'], [], ['Strategy', 'AWS', 'Figma', 'Google Cloud', 'Firebase', 'OOP']]


In [18]:
itemsets, rules = apriori(records, min_support=0.01, min_confidence=0.5, max_length=15)

In [24]:
result_apr_df = pd.DataFrame(columns=['antecedent', 'consequent', 'basket_rule', 'support', 'confidence', 'lift'])
for rule in rules:
    antecedent = list(rule.lhs)
    consequent = list(rule.rhs)
    basket_rule = str(antecedent) + "->" + str(consequent)
    support = rule.support
    confidence = rule.confidence
    lift = rule.lift
    result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)

result_apr_df = result_apr_df.sort_values(by=['confidence'], ascending=False)
result_apr_df.head()

C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  r

,antecedent,consequent,basket_rule,support,confidence,lift
1645,"[MVVM, TDD, iOS]",[RxSwift],"['MVVM', 'TDD', 'iOS']->['RxSwift']",0.011710,1.0,85.400000
1653,"[MVVM, RxSwift]","[TDD, iOS]","['MVVM', 'RxSwift']->['TDD', 'iOS']",0.011710,1.0,71.166667
1677,"[MySQL, Python, Redis]",[kafka],"['MySQL', 'Python', 'Redis']->['kafka']",0.021077,1.0,30.500000
1676,"[MySQL, Python, kafka]",[Redis],"['MySQL', 'Python', 'kafka']->['Redis']",0.021077,1.0,28.466667
1674,"[Python, Redis, kafka]",[MySQL],"['Python', 'Redis', 'kafka']->['MySQL']",0.021077,1.0,21.350000


In [25]:
result_apr_df = result_apr_df.rename_axis('index')

In [26]:
result_apr_df.to_csv('results.csv')

Load CSV files into MongoDB

In [27]:
import csv
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://35.171.48.20:27017')
db = client['IS459']
collection = db['analysis_results']

# Open CSV file and read rows into a list
with open('results.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

# Define the filter to check if the document already exists
for row in rows:
    filter = {'antecedent': row['antecedent'], 'consequent': row['consequent']}
    
    # Define the update operation to set the document if it doesn't exist
    update = {'$setOnInsert': row}
    
    # Execute the update operation with upsert=True to insert the document if it doesn't exist
    result = collection.update_one(filter, update, upsert=True)
    
    if result.upserted_id is not None:
        print(f"Document for antecedent: {row['antecedent']} and consequent: {row['consequent']} inserted.")
    else:
        print(f"Document for antecedent: {row['antecedent']} and consequent: {row['consequent']} already exists.")


Document for antecedent: ['MVVM', 'TDD', 'iOS'] and consequent: ['RxSwift'] inserted.
Document for antecedent: ['MVVM', 'RxSwift'] and consequent: ['TDD', 'iOS'] inserted.
Document for antecedent: ['MySQL', 'Python', 'Redis'] and consequent: ['kafka'] inserted.
Document for antecedent: ['MySQL', 'Python', 'kafka'] and consequent: ['Redis'] inserted.
Document for antecedent: ['Python', 'Redis', 'kafka'] and consequent: ['MySQL'] inserted.
Document for antecedent: ['Go', 'Python', 'Redis', 'Strategy', 'kafka'] and consequent: ['MySQL'] inserted.
Document for antecedent: ['MySQL', 'Python', 'Strategy'] and consequent: ['Redis'] inserted.
Document for antecedent: ['Python', 'Redis', 'Strategy'] and consequent: ['MySQL'] inserted.
Document for antecedent: ['MVVM', 'Swift'] and consequent: ['TDD', 'iOS'] inserted.
Document for antecedent: ['MVVM', 'iOS'] and consequent: ['Swift', 'TDD'] inserted.
Document for antecedent: ['MVVM', 'Swift', 'TDD'] and consequent: ['iOS'] inserted.
Document for

In [29]:
jobs_df.head()

,Unnamed: 0,company,title,stacks,salary,description
0,0,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
1,1,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
2,2,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
3,3,GXS Bank,"Software Engineer, Mobile (IOS)","[TDD, CI, RxSwift, MVVM, iOS, Swift]","['5,500', '8,250']",About The Team\n\nWhen people talk about Digib...
4,4,TikTok,Android Software Engineer - TikTok Live,"[Java, Android, Kotlin]","['6,787', '13,620']","Why Join Us\nAt TikTok, our people are humble,..."


In [50]:
stacks_df = pd.DataFrame(columns=['stack', 'count'])
stacks_map = {}

for row in jobs_df['stacks']:
    for item in row:
        if item not in stacks_map:
            stacks_map[item] = 1
        else:
            stacks_map[item] += 1
for stack, count in stacks_map.items():
    stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)

stacks_df

C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from

,stack,count
0,TDD,45
1,HTML,19
2,React Native,26
3,Android,27
4,Kotlin,13
...,...,...
254,WordPress,1
255,WebSphere,1
256,Pentaho,1
257,Data Mart,1


In [51]:
collection_stacks = db['stacks']

stack_records = stacks_df.to_dict('records')
collection_stacks.insert_many(stack_records)